# Лабораторная работа №7. Дискретное преобразование Фурье

### Лодочникова Владлена. Группа №5130901/10202

### Упражнение 7.1

В блокноте chap07.ipynb представлены дополнительные примеры и пояснения. Прочитали и запустили код. 

### Упражнение 7.2

В этой главе показано, как выразить ДПФ и обратное ДПФ произведениями
матриц. Время выполнения этих операций пропорционально
N^2, где N - длина массива сигнала. Во многих случаях этого
достаточно, но есть алгоритм и побыстрее - быстрое преобразование
Фурье (БПФ); время его работы пропорционально Nlog N.
Ключ к БПФ - это лемма Дэниелсона-Ланцоша (Danielson-
Lanczos).

Эта лемма предлагает рекурсивный алгоритм для ДПФ:
1. Дан массив сигнала у. Разделим его на четные элементы е и
нечетные элементы о.
2. Вычислим DFT е и о, делая рекурсивные вызовы.
3. Вычислим DFT(y) для каждого значения n, используя лемму
Дэниелсона-Ланцоша.

В простейшем случае эту рекурсию надо продолжать, пока длина у
не дойдет до 1. Тогда DFT(y) = у. А если длина у достаточно мала,
можно вычислить его ДПФ перемножением матриц, используя заранее
вычисленные матрицы.

Для сначала исследуем библиотечную фунукцию FFT. Для небольшого реального сигнала вычислим БПФ сигнал. 

In [1]:
import numpy as np
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


Создадим функцию для вычисления DFT. 

In [2]:
PI2 = 2 * np.pi

def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [3]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

Далее создадим функцию, которая делит массив пополам и вычисляет БПФ его половин. 

In [4]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [5]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Можно заметить, что результаты совпадают с ожидаемыми. 

Теперь заменим функцию np.fft.fft на рекурсивные вызовы и протестируем новую функцию вычисления БПФ

In [6]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

Проверим. 

In [7]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16

Функции FFT (Fast Fourier Transform) и DFT (Discrete Fourier Transform) используются для преобразования сигналов из временной области в частотную. Обе функции выполняют одно и то же преобразование, но FFT делает это быстрее, за счет использования алгоритма, основанного на разложениях в ряд Фурье. Время работы FFT составляет (n log (n)), в то время как для DFT оно равняется (n^2). Таким образом, FFT является более эффективным алгоритмом для вычисления преобразования Фурье для больших объемов данных.